In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, make_scorer
from sklearn.model_selection import GridSearchCV


In [43]:
x,y = load_breast_cancer(return_X_y=True, as_frame = True)


In [44]:
y = y.map({0:1, 1:0})

In [45]:
x.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [46]:
y.value_counts()/len(y)

,count
target,
0,0.627417
1,0.372583


In [47]:
def fnr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred,labels = [0,1]).ravel() #converts output to 1d array
    FNR = fn/(tp+fn)
    return FNR

In [48]:
fnr_score = make_scorer(fnr, greater_is_better = False,response_method = 'predict')# lower FNR values are better

In [49]:
rf_model = RandomForestClassifier(n_estimators=100,max_depth = 1, random_state=0,n_jobs =4) #n_jobs = use 4 cpu core.


In [50]:
rf_param_grid = dict(n_estimators=[10,20,50,100,200,500,1000,2000],max_depth = [1,2,3,4])

In [51]:
clf = GridSearchCV(rf_model, rf_param_grid, scoring=fnr_score, cv=5) #tests all hyperparamter and finds one with lowest fnr

In [52]:
search = clf.fit(x,y)

In [53]:
search.best_params_

{'max_depth': 3, 'n_estimators': 200}

In [54]:
# Select multiple columns by passing a list of column names
results = pd.DataFrame(search.cv_results_)[['params','mean_test_score','std_test_score']]

In [55]:
results.sort_values(by = 'mean_test_score', ascending = False, inplace = True)

In [56]:
results.reset_index(drop = True, inplace = True)

In [56]:
results['mean_test_score'].plot(yerr = [results['std_test_score'],results = ['std_test_score']] ,subplots =True)